In [1]:
import mysql.connector as con
import pandas as pd 
import numpy as np 
from surprise import Dataset, Reader
from surprise import KNNWithMeans
from surprise.model_selection import GridSearchCV

In [2]:
connection = con.connect(host='localhost', database='goodreads',user='mark',password='19991999')
if connection.is_connected():
    
    db_Info = connection.get_server_info()
    print("Connected to MySQL Server version ", db_Info)
    
    df = pd.read_sql("SELECT ratings.user_id, books.best_book_id, books.title, ratings.rating, books.average_rating FROM ratings Inner Join books on ratings.book_id = books.book_id", connection)


np.random.seed(10)

remove_n = 0
drop_indices = np.random.choice(df.index, remove_n, replace=False)
df = df.drop(drop_indices)
df

Connected to MySQL Server version  8.0.23-0ubuntu0.20.04.1


,user_id,best_book_id,title,rating,average_rating
0,1,1232,The Shadow of the Wind (The Cemetery of Forgot...,5,4.24
1,2,4865,How to Win Friends and Influence People,5,4.13
2,2,998,The Millionaire Next Door: The Surprising Secr...,3,4.00
3,2,968,"The Da Vinci Code (Robert Langdon, #2)",4,3.79
4,2,4894,Who Moved My Cheese?,3,3.74
...,...,...,...,...,...
5976474,49802,6098691,"Divine Misdemeanors (Merry Gentry, #8)",3,4.04
5976475,10622,25817162,America's First Daughter,4,4.22
5976476,35303,27824826,"The Dry (Aaron Falk, #1)",5,4.07
5976477,21879,7202841,"Locke & Key, Vol. 3: Crown of Shadows",5,4.41


In [3]:
reader = Reader(rating_scale=(1,5))
data = Dataset.load_from_df(df[["user_id", "best_book_id", "rating"]],reader)

In [4]:
sim_options = { "sim_options":{
    "name": ["msd","cosine"],
    "min_support": [3,4,5],
    "user_based": [False], 
}}
gs = GridSearchCV(KNNWithMeans, param_grid=sim_options, measures=["rmse","mae"], refit=True, cv=3,joblib_verbose=2, n_jobs=-1)


In [9]:
training = data.build_full_trainset()
gs.fit(data)
print(gs.best_score["rmse"])
print(gs.best_params["rmse"])

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
Computing the msd similarity matrix...
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   19.0s remaining:    0.0s
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarit

In [ ]:
prediction = algo.predict(2, 5)
print(df[(df["user_id"] == 2)  & (df["best_book_id"]==5)].title)
prediction